In [ ]:
# CNN Model And Optimisation Using Keras Tuner

In [ ]:
!pip install -U keras-tuner

In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd



In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zalando-research/fashionmnist")

print("Path to dataset files:", path)

In [9]:
df = keras.datasets.fashion_mnist

In [ ]:
df = df.load_data()

In [19]:
(trainingImages, trainingLabels), (testingImages, testingLabels) = df

In [20]:
# Scaling the Image Dataset
trainingImages = trainingImages/255.0
testingImages = testingImages/255.0

In [22]:
trainingImages[0].shape

(28, 28)

In [23]:
# Reshaping the Dataset
trainingImages = trainingImages.reshape(len(trainingImages), 28, 28, 1)
testingImages = testingImages.reshape(len(testingImages), 28, 28, 1)

In [25]:
trainingImages[0].shape

(28, 28, 1)

In [32]:
def buildModel(hp):
  model = keras.Sequential([
      # Input Layer
      keras.layers.Conv2D(
          filters = hp.Int('conv_1_filter', min_value = 16, max_value = 128, step = 16),
          kernel_size = hp.Choice('conv_1_kernel', values = [3, 5]),
          activation = 'relu',
          input_shape = (28, 28, 1)
      ),
      # Hidden Layer
      keras.layers.Conv2D(
          filters = hp.Int('conv_2_filter', min_value = 32, max_value = 64, step = 16),
          kernel_size = hp.Choice('conv_2_kernel', values = [3, 5]),
          activation = 'relu'
      ),
      # Output Layer
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int('dense_1_units', min_value = 32, max_value = 128, step = 16),
          activation = 'softmax'
      ),
  ])
  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values = [1e-2, 1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [35]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tunerSearch = RandomSearch(buildModel,
                           objective = 'val_accuracy',
                           max_trials = 5,
                           directory = 'output',
                           project_name = 'CNN MODEL')

In [39]:
tunerSearch.search(trainingImages, trainingLabels, epochs = 5, validation_split = 0.1)

Trial 5 Complete [00h 00m 59s]
val_accuracy: 0.8799999952316284

Best val_accuracy So Far: 0.9118333458900452
Total elapsed time: 00h 05m 06s


In [ ]:
model = tunerSearch.get_best_models(num_models = 1)[0]

In [41]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 80)     │           800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 22, 22, 32)     │        64,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 15488)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 96)             │     1,486,944 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,551,776 (5.92 MB)

 Trainable params: 1,551,776 (5.92 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
model.fit(trainingImages, trainingLabels, epochs=10, validation_split=0.1, initial_epoch=5)

Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9561 - loss: 0.1221 - val_accuracy: 0.9022 - val_loss: 0.3027
Epoch 7/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9660 - loss: 0.0953 - val_accuracy: 0.9150 - val_loss: 0.2938
Epoch 8/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9719 - loss: 0.0760 - val_accuracy: 0.9148 - val_loss: 0.3017
Epoch 9/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9792 - loss: 0.0612 - val_accuracy: 0.9087 - val_loss: 0.3408
Epoch 10/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9806 - loss: 0.0522 - val_accuracy: 0.9107 - val_loss: 0.4130
